# **Dataset**

Marital status: The marital status of the student. (Categorical) <br>
Application mode: The method of application used by the student. (Categorical)<br>
Application order: The order in which the student applied. (Numerical)<br>
Course: The course taken by the student. (Categorical)<br>
Daytime/evening attendance: Whether the student attends classes during the day or in the evening. (Categorical)<br>
Previous qualification: The qualification obtained by the student before enrolling in higher education. (Categorical)<br>
Nacionality: The nationality of the student. (Categorical)<br>
Mother's qualification: The qualification of the student's mother. (Categorical)<br>
Father's qualification: The qualification of the student's father. (Categorical)<br>
Mother's occupation: The occupation of the student's mother. (Categorical)<br>
Father's occupation: The occupation of the student's father. (Categorical)<br>
Displaced: Whether the student is a displaced person. (Categorical)<br>
Educational special needs: Whether the student has any special educational needs. (Categorical)<br>
Debtor: Whether the student is a debtor. (Categorical)<br>
Tuition fees up to date: Whether the student's tuition fees are up to date. (Categorical)<br>
Gender: The gender of the student. (Categorical)<br>
Scholarship holder: Whether the student is a scholarship holder. (Categorical)<br>
Age at enrollment: The age of the student at the time of enrollment. (Numerical)<br>
International: Whether the student is an international student. (Categorical)<br>
Curricular units 1st sem (credited): The number of curricular units credited by the student in the first semester. (Numerical)<br>
Curricular units 1st sem (enrolled): The number of curricular units enrolled by the student in the first semester. (Numerical)<br>
Curricular units 1st sem (evaluations): The number of curricular units evaluated by the student in the first semester. (Numerical)<br>
Curricular units 1st sem (approved): The number of curricular units approved by the student in the first semester. (Numerical)<br>

# **Install Libraries**

In [ ]:
!pip install scikit-learn
!pip install xgboost
!pip install featuretools
!pip install optuna

# **Import Packages**

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sb
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score,classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import featuretools as ft
import optuna
import matplotlib.pyplot as plt

# __Exploratory Data Analysis__

In [2]:
df = pd.read_csv('dataset.csv')
df.columns

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Displaced',
       'Educational special needs', 'Debtor', 'Tuition fees up to date',
       'Gender', 'Scholarship holder', 'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd sem (grade)',
       'Curricular units 2nd sem (without evaluations)

In [3]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 35 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Nacionality                                     4424 non-null   int64  
 7   Mother's qualification                          4424 non-null   int64  
 8   Father's qualification                          4424 non-null   int64  
 9   Mother's occupation                      

In [ ]:
countA, countB,countC = df.Target.value_counts()
print("Ratio of classes = ", countA, ":", countB, ":", countC)

# __Preprocessing__

In [4]:
# Rename columns
df.rename(columns={'Nacionality': 'Nationality'}, inplace=True)

features_list = ['Marital status', 
                 'Application mode',
                 'Application order',
                 'Course',
                 'Daytime/evening attendance',
                 'Previous qualification',
                 'Nationality',
                 "Mother's qualification",
                 "Father's qualification",
                 "Mother's occupation",
                 "Father's occupation",
                 'Displaced',
                 'Educational special needs',
                 'Debtor',
                 'Tuition fees up to date',
                 'Gender',
                 'Scholarship holder',
                 'Age at enrollment',
                 'International',
                 'Curricular units 1st sem (credited)',
                 'Curricular units 1st sem (enrolled)',
                 'Curricular units 1st sem (evaluations)',
                 'Curricular units 1st sem (approved)',
                 'Curricular units 1st sem (grade)',
                 'Curricular units 1st sem (without evaluations)',
                 'Curricular units 2nd sem (credited)',
                 'Curricular units 2nd sem (enrolled)',
                 'Curricular units 2nd sem (evaluations)',
                 'Curricular units 2nd sem (approved)',
                 'Curricular units 2nd sem (grade)',
                 'Curricular units 2nd sem (without evaluations)',
                 'Unemployment rate',
                 'Inflation rate',
                 'GDP']

forecast_var = ['Target']

In [5]:
df["Marital status"] = df["Marital status"].astype("category")
df["Application mode"] = df["Application mode"].astype("category")
df["Course"] = df["Course"].astype("category")
df["Daytime/evening attendance"] = df["Daytime/evening attendance"].astype("category")
df["Nationality"] = df["Nationality"].astype("category")
df["Mother's occupation"] = df["Mother's occupation"].astype("category")
df["Father's occupation"] = df["Father's occupation"].astype("category")
df["Displaced"] = df["Displaced"].astype("category")
df["Educational special needs"] = df["Educational special needs"].astype("category")
df["Debtor"] = df["Debtor"].astype("category")
df["Tuition fees up to date"] = df["Tuition fees up to date"].astype("category")
df["Gender"] = df["Gender"].astype("category")
df["Scholarship holder"] = df["Scholarship holder"].astype("category")
df["International"] = df["International"].astype("category")


## Creating Ordinal Data

Ordinal scale is where the order matters but not the difference between values. <br>
<br> Level of qualification can be ranked through an ordinal scale; however, it was not done so in the dataset. We hence need to reassign the numerical representations assigned to the qualification variables after ranking the various qualification levels provided.

Reference: https://www.graphpad.com/support/faq/what-is-the-difference-between-ordinal-interval-and-ratio-variables-why-should-i-care/ 

### Replacing and ranking the levels of qualifications under 'Previous qualification'

#### Initial list of variables for Previousqualification
1—Secondary education <br>
2—Higher education—bachelor’s degree <br>
3—Higher education—degree <br>
4—Higher education—master’s degree <br>
5—Higher education—doctorate <br>
6—Frequency of higher education <br>
7—12th year of schooling—not completed <br>
8—11th year of schooling—not completed <br>
9—Other—11th year of schooling <br>
10—10th year of schooling <br>
11—10th year of schooling—not completed <br>
12—Basic education 3rd cycle (9th/10th/11th year) or equivalent <br>
13—Basic education 2nd cycle (6th/7th/8th year) or equivalent <br>
14—Technological specialization course <br>
15—Higher education—degree (1st cycle) <br>
16—Professional higher technical course <br>
17—Higher education—master’s degree (2nd cycle)

#### Rearranged list of variables for Previousqualification

From https://en.wikipedia.org/wiki/Education_in_Portugal, high school = secondary education which constitutes years 10, 11 and 12. 

1 - Basic education 2nd cycle (6th/7th/8th year) or equivalent <br>
2 - 10th year of schooling—not completed <br>
3 - 10th year of schooling <br>
4 - 11th year of schooling—not completed <br>
5 - Other—11th year of schooling = Basic education 3rd cycle (9th/10th/11th year) or equivalent <br>
6 - 12th year of schooling—not completed <br>
7 - Secondary education <br>
8 - Frequency of higher education #assmue that higher education was incomplete <br>
9 - Technological specialization course <br>
10 - Professional higher technical course <br>
11 - Higher education—degree = Higher education—degree (1st cycle) <br>
12 - Higher education—bachelor’s degree <br>
13 - Higher education—master’s degree <br>
14 - Higher education—master’s degree (2nd cycle) <br>
15 - Higher education—doctorate <br>

In [6]:
#using key:value to replace values under 'Previous qualification'
previous_qualification_mapper = {1:7, 2:12, 3:11, 4:13, 5:15, 6:8, 7:6, 8:4, 9:5, 10:3, 11:2, 12:5, 13:1, 14:9, 15:11, 16:10, 17:14}
df["Previous qualification"] = df['Previous qualification'].replace(previous_qualification_mapper)


### Replacing and ranking the levels of qualifications under 'Mother's qualification' and 'Father's qualification'

#### Initial list of variables for Mother's qualification and Father's qualification 
1—Secondary Education—12th Year of Schooling or Equivalent <br>
2—Higher Education—bachelor’s degree <br>
3—Higher Education—degree <br>
4—Higher Education—master’s degree <br>
5—Higher Education—doctorate <br>
6—Frequency of Higher Education <br>
7—12th Year of Schooling—not completed <br>
8—11th Year of Schooling—not completed <br>
9—7th Year (Old) <br>
10—Other—11th Year of Schooling <br>
11—2nd year complementary high school course <br>
12—10th Year of Schooling <br>
13—General commerce course <br>
14—Basic Education 3rd Cycle (9th/10th/11th Year) or Equivalent <br>
15—Complementary High School Course <br>
16—Technical-professional course <br>
17—Complementary High School Course—not concluded <br>
18—7th year of schooling <br>
19—2nd cycle of the general high school course <br>
20—9th Year of Schooling—not completed <br>
21—8th year of schooling <br>
22—General Course of Administration and Commerce <br>
23—Supplementary Accounting and Administration <br>
24—Unknown <br>
25—Cannot read or write <br>
26—Can read without having a 4th year of schooling <br>
27—Basic education 1st cycle (4th/5th year) or equivalent <br>
28—Basic Education 2nd Cycle (6th/7th/8th Year) or equivalent <br>
29—Technological specialization course <br>
30—Higher education—degree (1st cycle) <br>
31—Specialized higher studies course <br>
32—Professional higher technical course <br>
33—Higher Education—master’s degree (2nd cycle) <br>
34—Higher Education—doctorate (3rd cycle) 

#### Rearranged list of variables for Mother's qualification and Father's qualification

0 - Unknown <br>
1 - Cannnot read or write <br>
2 - Can read without having a 4th year of schooling <br>
3 - Basic education 1st cycle (4th/5th year) or equivalent <br>
4 - Basic Education 2nd Cycle (6th/7th/8th Year) or equivalent <br>
5 - 7th year (Old) = 7th year of schooling <br>
6 - 8th year of schooling <br>
7 - 9th Year of Schooling—not completed <br>
8 - Complementary High School Course—not concluded <br> 
9 - 10th Year of Schooling <br>
10 - 11th Year of Schooling—not completed <br>
11 - Other—11th Year of Schooling = 2nd year complementary high school course = Basic Education 3rd Cycle (9th/10th/11th Year) or Equivalent <br>
12 - 12th Year of Schooling—not completed <br>
13 - Secondary Education—12th Year of Schooling or Equivalent = Complementary High School Course = 2nd cycle of the general high school course #assume multiple cycles of high school refers to retaking high school after completing it. <br>
14 - Frequency of Higher Education #assume that higher education was incomplete <br> 
15 - General commerce course = General Course of Administration and Commerce = Supplementary Accounting and Administration = Technical-professional course = Technological specialization course #assume these courses are merely general specialisation courses and part of higher education but does not constitute as degrees <br>
16 - Specialized higher studies course = Professional higher technical course #assume these are higher level specialised that does not constitute degrees <br>
17 - Higher Education—degree = Higher education—degree (1st cycle) #assume that 1st cycle means higher completed first higher education degree (eg double degree holders etc), we also assume that the unspecified degree refers to the lowest level of college associate degree https://thebestschools.org/degrees/college-degree-levels/ <br>
18 - Higher Education—bachelor’s degree <br>
19 - Higher Education—master’s degree <br>
20 - Higher Education—master’s degree (2nd cycle) <br>
21 - Higher Education—doctorate <br>
22 - Higher Education—doctorate (3rd cycle) <br>
                    

In [7]:
parent_qualification_mapper = {1:13, 2:18, 3:17, 4:19, 5:21, 6:14, 7:12, 8:10, 9:5, 10:11, 11:11, 12:9, 13:15, 14:11, 15:13, 16:15, 17:8, 18:5, 19:13, 20:7, 21:6, 22:15, 23:15, 24:0, 25:1, 26:2, 27:3, 28:4, 29:15, 30:17, 31:16, 32:16, 33:20, 34:22}
df["Mother's qualification"] = df["Mother's qualification"].replace(parent_qualification_mapper)
df["Father's qualification"] = df["Father's qualification"].replace(parent_qualification_mapper)

#### Imputing unknown values in 'Mother's qualification', 'Father's qualification', 'Mother's occupation', 'Father's occupation'
Mean imputation is often used when the missing values are numerical and the distribution of the variable is approximately normal. <br>
Median imputation is preferred when the distribution is skewed, as the median is less sensitive to outliers than the mean.<br>
Mode imputation is suitable for categorical variables or numerical variables with a small number of unique values.<br>
Hence, missing values should be imputed with the mode for this dataset.

#### Find skew and mode of each feature

In [ ]:
print('Mode: ',df["Mother's qualification"].mode())
print('Mode: ',df["Father's qualification"].mode())
print('Mode: ',df["Mother's occupation"].mode())
print('Mode: ',df["Father's occupation"].mode())

sb.catplot(y = "Mother's qualification", data = df, kind = "count")
sb.catplot(y= "Father's qualification", data = df, kind = 'count')
sb.catplot(y = "Mother's occupation", data = df, kind = "count")
sb.catplot(y= "Father's occupation", data = df, kind = 'count')

plt.show()

In [8]:
#replace features 'Unknown' with the mode
df["Mother's qualification"] = df["Mother's qualification"].replace({24:15})
df["Father's qualification"] = df["Father's qualification"].replace({24:3})

#replace features 'Other Situation' and '(blank)' with the mode
df["Mother's occupation"] = df["Mother's occupation"].replace({12:10, 13:10})
df["Father's occupation"] = df["Father's occupation"].replace({12:10, 13:10})

In [10]:
#Save preprocessed dataset
df.to_csv("clean_dataset_zon.csv", index=False )

In [11]:
""""""
from sklearn.preprocessing import StandardScaler

#Normalise numerical features to reduce differences between values of different columns
ss = StandardScaler()
numerical_features = ['Age at enrollment', 'Curricular units 1st sem (credited)',
                 'Curricular units 1st sem (enrolled)',
                 'Curricular units 1st sem (evaluations)',
                 'Curricular units 1st sem (approved)',
                 'Curricular units 1st sem (grade)',
                 'Curricular units 1st sem (without evaluations)',
                 'Curricular units 2nd sem (credited)',
                 'Curricular units 2nd sem (enrolled)',
                 'Curricular units 2nd sem (evaluations)',
                 'Curricular units 2nd sem (approved)',
                 'Curricular units 2nd sem (grade)',
                 'Curricular units 2nd sem (without evaluations)',
                 'Unemployment rate',
                 'Inflation rate',
                 'GDP',"Mother's qualification","Father's qualification","Previous qualification"]
df[numerical_features] = ss.fit_transform(df[numerical_features])



## Feature Engineering w FeatureTools

In [12]:
from featuretools.primitives import *

In [13]:
# Add tracker column to track index
df.insert(0, 'Tracker', range(4424))

In [14]:
# create an entity set 'es'
es = ft.EntitySet(id = 'Target')

# adding a dataframe 
es = es.add_dataframe(
    dataframe_name="students",
    dataframe=df,
    index="Tracker",
)

### Run Deep Feature Synthesis (DFS)

In [21]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_dataframe_name = 'students',
                                       max_depth = 1, verbose = 1, n_jobs = 3)

Built 35 features
EntitySet scattered to 3 workers in 9 seconds                                                                          
Elapsed: 00:00 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [22]:
feature_matrix.columns.reindex(df['Tracker'])
feature_matrix = feature_matrix.reset_index()
feature_matrix.head()

,Tracker,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nationality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,8,5,2,1,-0.182347,1,0.206340,0.493130,...,-0.282442,-2.838337,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761,Dropout
1,1,1,6,1,11,1,-0.182347,1,-0.848976,1.645184,...,-0.282442,-0.105726,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199,Graduate
2,2,1,1,5,5,1,-0.182347,1,0.206340,-1.042942,...,-0.282442,-0.105726,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761,Dropout
3,3,1,8,2,15,1,-0.182347,1,0.206340,-1.042942,...,-0.282442,-0.105726,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511,Graduate
4,4,2,12,1,3,0,-0.182347,1,0.206340,-0.850933,...,-0.282442,-0.105726,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199,Graduate


In [23]:
#feature_matrix.drop(['Tracker'], axis=1, inplace=True)
X = feature_matrix.iloc[:,:35]
y = feature_matrix.iloc[:,35:]

In [24]:
#Drop features with no importance: Previous qualification, Daytime/evening attendance, International 
#df.drop(['Previous qualification','Daytime/evening attendance','International'], axis = 1)
#X = df[features_list]
#y = df[forecast_var] 

# Encode categorical features
y = y.astype("category")
le = LabelEncoder()
y = le.fit_transform(np.ravel(y))

# **Hyperparameter Tuning w Optuna**

In [25]:
#hide warnings so they do not affect the functionality of the package
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)
# Split validation set from initial train set to form 8:1:1 train:validation:test ratio
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.89,)

In [31]:
type(dtrain)

xgboost.core.DMatrix

In [52]:
from sklearn.metrics import log_loss

def objective_with_pruning(trial):
    #define the hyperparameters to optimize
    params = {
        'eval_metric': 'mlogloss',
        'objective': 'multi:softmax',
        'max_depth': trial.suggest_int('max_depth', 5, 8),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 1800, 4000, 50),
        'eta': trial.suggest_loguniform('eta', 0.005, 0.1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1e-5),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-6, 0.5),
        'min_split_loss': trial.suggest_loguniform('gamma', 1e-5, 1), 
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 0.5)}
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-mlogloss")
    model = xgb.XGBClassifier(early_stopping_rounds = 50, booster = 'gbtree',tree_method= 'approx',enable_categorical=True)
    model.set_params(**params)
    model.fit(X_train,y_train, eval_set = [(X_val,y_val)], callbacks=[pruning_callback], verbose = 10) 
    
    preds = model.predict_proba(X_val)
    loss = log_loss(y_val, preds)
    return loss

In [53]:
"""
1. 100 trials
Trial 72 finished with value: 0.552958542544134 and parameters: 
{'max_depth': 5, 'subsample': 0.8, 'n_estimators': 1950, 'eta': 0.02935051557466342, 
'reg_alpha': 2.7452323045088246e-08, 'reg_lambda': 3.771084614288098e-05, 'gamma': 0.000392937067496624,
'min_child_weight': 12, 'colsample_bytree': 0.2739207213317758}. 

2.500 trials
Trial 244 finished with value: 0.5555424854967693 and parameters: 
{'max_depth': 6, 'subsample': 0.95, 'n_estimators': 3350, 'eta': 0.008666006292274537, 
'reg_alpha': 1.2870589813811533e-06, 'reg_lambda': 0.13154643432513044, 'gamma': 0.026360214147244408
'min_child_weight': 5, 'colsample_bytree': 0.22708548202884274}. 

3. 100 trials, changed parameter search space, added pruning callback
Trial 22 finished with value: 0.5520506186121606 and parameters: 
{'max_depth': 6, 'subsample': 0.65, 'n_estimators': 2500, 'eta': 0.027735798765723383, 
'reg_alpha': 2.8442623168179127e-08, 'reg_lambda': 9.97186254008275e-05, 'gamma': 0.00042214727119007766, 
'min_child_weight': 11, 'colsample_bytree': 0.38249471476294605}.

4. 500 trials
Trial 217 finished with value: 0.5511272925050019 and parameters: 
{'max_depth': 7, 'subsample': 0.8, 'n_estimators': 3950, 'eta': 0.049656985841559836, 
'reg_alpha': 1.9886858590626805e-08, 'reg_lambda': 8.750237693599536e-05, 'gamma': 0.02578091778103115, 
'min_child_weight': 14, 'colsample_bytree': 0.3834674148298466}.

5. Trial 49 finished with value: 0.5502489012619268 and parameters: 
{'max_depth': 7, 'subsample': 0.7, 'n_estimators': 3800, 'eta': 0.09967223617945833, 
'reg_alpha': 4.1181276209901314e-08, 'reg_lambda': 0.040098666589886055, 'gamma': 0.924533874823556, 
'min_child_weight': 5, 'colsample_bytree': 0.33274357291224116}
Accuracy:0.78, F1 Score:[0.77 0.46  0.87]
"""
study = optuna.create_study(direction='minimize')
study.optimize(objective_with_pruning, n_trials=100) #change number of trials later

[I 2023-03-29 13:57:14,034] A new study created in memory with name: no-name-694a23af-4bd9-4c55-98b9-49ed3bfdb459


[0]	validation_0-mlogloss:1.09354
[10]	validation_0-mlogloss:1.04707
[20]	validation_0-mlogloss:1.00211
[30]	validation_0-mlogloss:0.96300
[40]	validation_0-mlogloss:0.92815
[50]	validation_0-mlogloss:0.89649
[60]	validation_0-mlogloss:0.86808
[70]	validation_0-mlogloss:0.84245
[80]	validation_0-mlogloss:0.81943
[90]	validation_0-mlogloss:0.79717
[100]	validation_0-mlogloss:0.77775
[110]	validation_0-mlogloss:0.76050
[120]	validation_0-mlogloss:0.74560
[130]	validation_0-mlogloss:0.73075
[140]	validation_0-mlogloss:0.71750
[150]	validation_0-mlogloss:0.70434
[160]	validation_0-mlogloss:0.69358
[170]	validation_0-mlogloss:0.68325
[180]	validation_0-mlogloss:0.67408
[190]	validation_0-mlogloss:0.66521
[200]	validation_0-mlogloss:0.65765
[210]	validation_0-mlogloss:0.64991
[220]	validation_0-mlogloss:0.64310
[230]	validation_0-mlogloss:0.63636
[240]	validation_0-mlogloss:0.63078
[250]	validation_0-mlogloss:0.62540
[260]	validation_0-mlogloss:0.62044
[270]	validation_0-mlogloss:0.61609
[28

[I 2023-03-29 13:59:12,576] Trial 0 finished with value: 0.5571405587571175 and parameters: {'max_depth': 8, 'subsample': 0.9, 'n_estimators': 3900, 'eta': 0.008365103949136306, 'reg_alpha': 7.410669113021105e-08, 'reg_lambda': 0.3843956956709221, 'gamma': 0.03983310447516707, 'min_child_weight': 7, 'colsample_bytree': 0.4152706258048521}. Best is trial 0 with value: 0.5571405587571175.


[0]	validation_0-mlogloss:1.08840
[10]	validation_0-mlogloss:1.00007
[20]	validation_0-mlogloss:0.92643
[30]	validation_0-mlogloss:0.86874
[40]	validation_0-mlogloss:0.82035
[50]	validation_0-mlogloss:0.78507
[60]	validation_0-mlogloss:0.75506
[70]	validation_0-mlogloss:0.72727
[80]	validation_0-mlogloss:0.70462
[90]	validation_0-mlogloss:0.68399
[100]	validation_0-mlogloss:0.66736
[110]	validation_0-mlogloss:0.65391
[120]	validation_0-mlogloss:0.64234
[130]	validation_0-mlogloss:0.63211
[140]	validation_0-mlogloss:0.62382
[150]	validation_0-mlogloss:0.61573
[160]	validation_0-mlogloss:0.61011
[170]	validation_0-mlogloss:0.60419
[180]	validation_0-mlogloss:0.59982
[190]	validation_0-mlogloss:0.59465
[200]	validation_0-mlogloss:0.59093
[210]	validation_0-mlogloss:0.58770
[220]	validation_0-mlogloss:0.58507
[230]	validation_0-mlogloss:0.58199
[240]	validation_0-mlogloss:0.57990
[250]	validation_0-mlogloss:0.57829
[260]	validation_0-mlogloss:0.57615
[270]	validation_0-mlogloss:0.57494
[28

[I 2023-03-29 14:00:38,294] Trial 1 finished with value: 0.5562613600446124 and parameters: {'max_depth': 6, 'subsample': 0.9, 'n_estimators': 3600, 'eta': 0.018503924224253954, 'reg_alpha': 9.32868321616411e-08, 'reg_lambda': 0.19890153457209536, 'gamma': 0.004900517560155265, 'min_child_weight': 8, 'colsample_bytree': 0.2987209922107936}. Best is trial 1 with value: 0.5562613600446124.


...

[I 2023-03-29 14:10:02,945] Trial 96 pruned. Trial was pruned at iteration 0.
[I 2023-03-29 14:10:03,127] Trial 97 pruned. Trial was pruned at iteration 0.


[0]	validation_0-mlogloss:1.05312
[10]	validation_0-mlogloss:0.76655
[20]	validation_0-mlogloss:0.66513
[30]	validation_0-mlogloss:0.61568
[40]	validation_0-mlogloss:0.59364
[50]	validation_0-mlogloss:0.58208
[60]	validation_0-mlogloss:0.57316
[70]	validation_0-mlogloss:0.56817
[80]	validation_0-mlogloss:0.56239
[90]	validation_0-mlogloss:0.55830
[100]	validation_0-mlogloss:0.55947
[110]	validation_0-mlogloss:0.56007
[120]	validation_0-mlogloss:0.55668
[130]	validation_0-mlogloss:0.55825
[140]	validation_0-mlogloss:0.55985
[150]	validation_0-mlogloss:0.56220
[160]	validation_0-mlogloss:0.56384
[170]	validation_0-mlogloss:0.56613
[171]	validation_0-mlogloss:0.56598


[I 2023-03-29 14:10:11,934] Trial 98 finished with value: 0.556330393610435 and parameters: {'max_depth': 7, 'subsample': 0.85, 'n_estimators': 3750, 'eta': 0.09994074015308665, 'reg_alpha': 1.481474233924668e-07, 'reg_lambda': 0.3523683310609304, 'gamma': 0.1285048751486935, 'min_child_weight': 6, 'colsample_bytree': 0.2801881217273825}. Best is trial 49 with value: 0.5502489012619268.


[0]	validation_0-mlogloss:1.06711


[I 2023-03-29 14:10:12,128] Trial 99 pruned. Trial was pruned at iteration 0.


In [54]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.5502489012619268
  Params: 
    max_depth: 7
    subsample: 0.7
    n_estimators: 3800
    eta: 0.09967223617945833
    reg_alpha: 4.1181276209901314e-08
    reg_lambda: 0.040098666589886055
    gamma: 0.924533874823556
    min_child_weight: 5
    colsample_bytree: 0.33274357291224116


In [55]:
# Train new model on best params from optuna and evaluate performance
best_params = study.best_params
model = xgb.XGBClassifier(early_stopping_rounds = 50, booster = 'gbtree',tree_method= 'approx',enable_categorical=True)
model.set_params(**best_params)
model.fit(X_train,y_train, eval_set = [(X_val,y_val)], verbose = 10) 
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

[0]	validation_0-mlogloss:1.04368
[10]	validation_0-mlogloss:0.75171
[20]	validation_0-mlogloss:0.64817
[30]	validation_0-mlogloss:0.60464
[40]	validation_0-mlogloss:0.57999
[50]	validation_0-mlogloss:0.56778
[60]	validation_0-mlogloss:0.56099
[70]	validation_0-mlogloss:0.55547
[80]	validation_0-mlogloss:0.55328
[90]	validation_0-mlogloss:0.55295
[100]	validation_0-mlogloss:0.55190
[110]	validation_0-mlogloss:0.55044
[120]	validation_0-mlogloss:0.55127
[130]	validation_0-mlogloss:0.55265
[140]	validation_0-mlogloss:0.55329
[150]	validation_0-mlogloss:0.55596
[154]	validation_0-mlogloss:0.55651
              precision    recall  f1-score   support

           0       0.80      0.75      0.77       138
           1       0.59      0.38      0.46        80
           2       0.81      0.95      0.87       225

    accuracy                           0.78       443
   macro avg       0.73      0.69      0.70       443
weighted avg       0.77      0.78      0.77       443



In [56]:
optuna.visualization.plot_param_importances(study)

In [57]:
optuna.visualization.plot_optimization_history(study)

# **Multi-Class Classification with XGBoost**

# **Create Model**

In [ ]:
"""
Experiments:
#Zon's dataset (no ordinal features)
1. n_estimators = 5000 (overfit beyond 3900), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True
logloss:0.55598, Accuracy:0.7742663656884876, F1 Score:[0.76760563 0.4822695  0.86767896]
2. Training Time of Several Hours: n_estimators = 3900 (overfit beyond 2990), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'dart',enable_categorical=True
logloss:0.59134, Accuracy:0.7652370203160271, F1 Score:[0.79856115 0.49350649 0.83700441]
#Wynette's dataset (ordinal features)
3. n_estimators = 3900 (overfit beyond 3400), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True
logloss:0.55960, Accuracy:0.77, F1 Score:[0.77 0.45  0.85]
4. # drop features with lowest feature_importance score: Previous qualification, Daytime/evening attendance, International
n_estimators = 3900 (overfit beyond 2920), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True
logloss:0.61181, Accuracy:0.78, F1 Score:[0.76 0.48  0.87]

5. # drop features + scaling of numerical features
n_estimators = 5000 (overfit beyond 3980), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True
logloss:0.53519, Accuracy:0.77, F1 Score:[0.78 0.43  0.86]

6. # scaling of numerical features
n_estimators = 5000 (overfit beyond 2700), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True
logloss:0.61050, Accuracy:0.71, F1 Score:[0.71 0.41  0.83]

7. # feature engineering w FeatureTools
n_estimators = 5000 (overfit beyond 3860), early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True
logloss:0.52097, Accuracy:0.75, F1 Score:[0.77 0.39  0.83]
"""
# Logloss better metric for training than accuracy/F1 score. 
# Logloss measures model skill, accuracy/F1 score measures performance on test set, which varies.
model = xgb.XGBClassifier(n_estimators = 5000, early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'gbtree',enable_categorical=True)
#model.load_model('model_1.json')
model.fit(X_train, y_train, eval_set = [(X_val,y_val)], verbose = 10)

In [59]:
model.save_model('model_3.json')

In [58]:
# Rank features by importance
df_feature_importance = pd.DataFrame(index = model.get_booster().feature_names, data = model.feature_importances_, columns = ['Importance'])
df_feature_importance.sort_values('Importance',ascending=False)

,Importance
Tuition fees up to date,0.106407
Curricular units 2nd sem (approved),0.086541
Curricular units 1st sem (approved),0.078598
Curricular units 2nd sem (grade),0.052596
Scholarship holder,0.036215
Curricular units 1st sem (evaluations),0.035013
Curricular units 2nd sem (enrolled),0.034551
Course,0.034456
Debtor,0.034421
Curricular units 1st sem (enrolled),0.033152


If a feature has a low feature_importance, it doesn’t mean that this feature is uninformative. It only means that the feature was not picked by the tree, likely because another feature encodes the same information.

Drop features with no importance
Train model on new dataset and evaluate

# **Predict and Evaluate Performance**

In [ ]:
# Use model to predict on test data
y_test_pred = model.predict(X_test)
df.head()

# Print the Classification Accuracy
print("Test Data")

#print("Accuracy  :\t", model.score(X_test, y_test))
# Dropout, Enrolled, Graduate
#print("F1 Score  :\t",f1_score(y_test, y_test_pred, average=None))

print(classification_report(y_test, y_test_pred))

# Plot the two-way Confusion Matrix
plt.ylabel('Actual')
plt.xlabel('Predicted')
sb.heatmap(confusion_matrix(y_test, y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18})
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# **Visualise Tree**

In [ ]:
# Visualise first tree of model
fig = xgb.plot_tree(model)
plt.savefig('XGB_tree_3.png', dpi = 4800)

# **Anomaly Detection with Isolation Forest**

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
df = pd.read_csv('dataset.csv')

# **Create Model**

In [ ]:
model_IF = IsolationForest(n_estimators=5000, contamination = 0.32, max_features= 34, verbose = 10, bootstrap=True)
model_IF.fit(X.values)

# **Predict and Evaluate Performance**

In [ ]:
# model.load_model('model_IF_1.json')
# predict on testset
y_test_pred = model_IF.predict(X_test.values)
y_test_pred = pd.Series(y_test_pred)

# merge 'Graduate' and 'Enrolled' classes in y_test
y_test = pd.Series(y_test).replace(to_replace=2, value=1)
# 0:outlier (Dropout), 1: inlier
y_test_pred = y_test_pred.map({1: 1, -1: 0})

print(classification_report(y_test, y_test_pred))
# Plot the two-way Confusion Matrix
sb.heatmap(confusion_matrix(y_test, y_test_pred),annot = True, fmt=".0f", annot_kws={"size": 18})
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()    

In [ ]:
model.save_model('model_IF_1.json')